![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Legal Deidentification

In [0]:
from johnsnowlabs import *

# Deidentification Model

Some legal information can be considered sensitive. (e.g.,document, organization, address, signer)

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
    #.setCustomBounds(["\n\n"])

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

legal_ner = legal.NerModel.pretrained("legner_contract_doc_parties_lg", "en", "legal/models")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner") 
    #.setLabelCasing("upper")

ner_converter = legal.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setReplaceLabels({"ALIAS": "PARTY"}) # "ALIAS" are secondary names of companies, so let's extract them also as PARTY

nlpPipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
legner_contract_doc_parties_lg download started this may take some time.
[ | ][ / ][ — ][OK!]

### Pretrained NER models extracts:
- Document
- Date
- Party (Organization Name)
- Alias

In [0]:
legal_ner.getClasses()

Out[3]: ['O',
 'B-ORG',
 'I-DOC',
 'B-EFFDATE',
 'I-ORG',
 'B-ALIAS',
 'I-ALIAS',
 'B-PARTY',
 'I-EFFDATE',
 'B-FORMER_NAME',
 'I-FORMER_NAME',
 'B-DOC',
 'I-PARTY']

In [0]:
text = """THIS STRATEGIC ALLIANCE AGREEMENT ("Agreement") is made and entered into as of December 14, 2016 , by and between Hyatt Franchising Latin America, L.L.C. a limited liability company organized and existing under the laws of the State of Delaware"""

In [0]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [0]:
from pyspark.sql import functions as F

result_df = result.select(F.explode(F.arrays_zip(result.token.result, 
                                                 result.ner.result)).alias("cols")) \
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("ner_label"))

In [0]:
result_df.select("token", "ner_label").groupBy('ner_label').count().orderBy('count', ascending=False).show(truncate=False)

+---------+-----+
ner_label|count|
+---------+-----+
O |31 |
I-PARTY |5 |
I-EFFDATE|3 |
I-DOC |2 |
B-DOC |1 |
B-PARTY |1 |
B-EFFDATE|1 |
+---------+-----+

### Check extracted sensitive entities

In [0]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+--------------------------------------+---------+
chunk |ner_label|
+--------------------------------------+---------+
STRATEGIC ALLIANCE AGREEMENT |DOC |
December 14, 2016 |EFFDATE |
Hyatt Franchising Latin America, L.L.C|PARTY |
+--------------------------------------+---------+

## Masking and Obfuscation

### Replace these enitites with Tags

In [0]:
deidentification = legal.DeIdentification() \
      .setInputCols(["sentence", "token", "ner_chunk"]) \
      .setOutputCol("deidentified") \
      .setMode("mask")\
      .setReturnEntityMappings(True) #  return a new column to save the mappings between the mask/obfuscated entities and original entities. REquired for "ReIdentification"
      #.setMappingsColumn("MappingCol") # change the name of the column, 'aux' is default

deidPipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter,
      deidentification])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model_deid = deidPipeline.fit(empty_data)

In [0]:
result = model_deid.transform(spark.createDataFrame([[text]]).toDF("text"))

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| deidentified| aux|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
THIS STRATEGIC AL...|[{document, 0, 24...|[{document, 0, 24...|[{token, 0, 3, TH...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 5, 32, S...|[{document, 0, 18...|[{chunk, 5, 9, <D...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
reIdentification = legal.ReIdentification()\
    .setInputCols(["aux","deidentified"])\
    .setOutputCol("original")

In [0]:
reid_result = reIdentification.transform(result)

In [0]:
reid_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| deidentified| aux| original|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
THIS STRATEGIC AL...|[{document, 0, 24...|[{document, 0, 24...|[{token, 0, 3, TH...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 5, 32, S...|[{document, 0, 18...|[{chunk, 5, 9, <D...|[{document, 0, 24...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

# ReIdentification

In [0]:
print(text)

reid_result.select('original.result').show(truncate=False)

THIS STRATEGIC ALLIANCE AGREEMENT ("Agreement") is made and entered into as of December 14, 2016 , by and between Hyatt Franchising Latin America, L.L.C. a limited liability company organized and existing under the laws of the State of Delaware
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
result |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[THIS STRATEGIC ALLIANCE AGREEMENT ("Agreement") is made and entered into as of December 14, 2016 , by and between Hyatt Franchising Latin America, L.L.C. a limited liability company organized and existing under the laws of the State of Delaware]|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [0]:
result.select(F.explode(F.arrays_zip(result.sentence.result, 
                                     result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), 
              F.expr("cols['1']").alias("deidentified")).toPandas()

Out[16]:

,sentence,deidentified
0,"THIS STRATEGIC ALLIANCE AGREEMENT (""Agreement""...","THIS <DOC> (""Agreement"") is made and entered i..."


## Other different masking strategies

We have three modes to mask the entities in the Deidentification annotator. You can select the modes using the `.setMaskingPolicy()` parameter. The methods are the followings:

**“entity_labels”**: Mask with the entity type of that chunk. (default) <br/>
**“same_length_chars”**: Mask the deid entities with same length of asterix ( * ) with brackets ( [ , ] ) on both end. <br/>
**“fixed_length_chars”**: Mask the deid entities with a fixed length of asterix ( * ). The length is setting up using the `setFixedMaskLength()` method. <br/>

Let's try each of these and compare the results:

In [0]:
#deid model with "entity_labels"
deid_entity_labels= legal.DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"])\
    .setOutputCol("deid_entity_label")\
    .setMode("mask")\
    .setReturnEntityMappings(False)\
    .setMaskingPolicy("entity_labels")

#deid model with "same_length_chars"
deid_same_length= legal.DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"])\
    .setOutputCol("deid_same_length")\
    .setMode("mask")\
    .setReturnEntityMappings(False)\
    .setMaskingPolicy("same_length_chars")

#deid model with "fixed_length_chars"
deid_fixed_length= legal.DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"])\
    .setOutputCol("deid_fixed_length")\
    .setMode("mask")\
    .setReturnEntityMappings(False)\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)


deidPipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter,
      deid_entity_labels,
      deid_same_length,
      deid_fixed_length])


empty_data = spark.createDataFrame([[""]]).toDF("text")
model_deid = deidPipeline.fit(empty_data)

In [0]:
result = model_deid.transform(spark.createDataFrame([[text]]).toDF("text"))

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| deid_entity_label| deid_same_length| deid_fixed_length|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
THIS STRATEGIC AL...|[{document, 0, 24...|[{document, 0, 24...|[{token, 0, 3, TH...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 5, 32, S...|[{document, 0, 18...|[{document, 0, 24...|[{document, 0, 17...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
result.select(F.explode(F.arrays_zip(result.sentence.result, 
                                            result.deid_entity_label.result, 
                                            result.deid_same_length.result, 
                                            result.deid_fixed_length.result)).alias("cols")) \
             .select(F.expr("cols['0']").alias("sentence"),
                     F.expr("cols['1']").alias("deid_entity_label"),
                     F.expr("cols['2']").alias("deid_same_length"),
                     F.expr("cols['3']").alias("deid_fixed_length")).toPandas()

Out[20]:

,sentence,deid_entity_label,deid_same_length,deid_fixed_length
0,"THIS STRATEGIC ALLIANCE AGREEMENT (""Agreement""...","THIS <DOC> (""Agreement"") is made and entered i...","THIS [**************************] (""Agreement""...","THIS **** (""Agreement"") is made and entered in..."


### Mapping Column

In [0]:
result.select("ner_chunk").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
ner_chunk |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[{chunk, 5, 32, STRATEGIC ALLIANCE AGREEMENT, {chunk -> 0, confidence -> 0.9776667, ner_source -> ner_chunk, entity -> DOC, sentence -> 0}, []}, {chunk, 79, 95, December 14, 2016, {chunk -> 1, confidence -> 0.99065, ner_source -> ner_chunk, entity -> EFFDATE, sentence -> 0}, []}, {chunk, 114, 151, Hyatt Franchising Latin America, L.L.C, {chunk -> 2, confidence -> 0.9488333, ner_source -> ner_chunk, entity -> PARTY, sentence -> 0}, []}]|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [0]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.metadata,
                                     result.ner_chunk.begin, 
                                     result.ner_chunk.end)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['2']").alias("begin"),
              F.expr("cols['3']").alias("end"),
              F.expr("cols['1']['entity']").alias("label"),
              F.expr("cols['2']").alias("beginLabel"),
              F.expr("cols['3']").alias("endLabel")).show(truncate=False)

+--------------------------------------+-----+---+-------+----------+--------+
chunk |begin|end|label |beginLabel|endLabel|
+--------------------------------------+-----+---+-------+----------+--------+
STRATEGIC ALLIANCE AGREEMENT |5 |32 |DOC |5 |32 |
December 14, 2016 |79 |95 |EFFDATE|79 |95 |
Hyatt Franchising Latin America, L.L.C|114 |151|PARTY |114 |151 |
+--------------------------------------+-----+---+-------+----------+--------+

## Using NER, ContextualParser and ZeroShotNER in the same Deideintification pipeline

In [0]:
# Create JSON file for PART
alias = {
  "entity": "ALIAS",
  "ruleScope": "document", 
  "completeMatchRegex": "true",
  "regex":'["“].*?["”]',
  "matchScope": "sub-token",
  "contextLength": 100
}

email = {
  "entity": "EMAIL",
  "ruleScope": "document", 
  "completeMatchRegex": "true",
  "regex":'[\w-\.]+@([\w-]+\.)+[\w-]{2,4}',
  "matchScope": "sub-token",
  "contextLength": 100
}

phone = {
  "entity": "PHONE",
  "ruleScope": "document", 
  "completeMatchRegex": "true",
  "regex":'(\+?\d{1,3}[\s-]?)?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d+',
  "matchScope": "sub-token",
  "contextLength": 100
}

import json
with open('alias.json', 'w') as f:
    json.dump(alias, f)
    
with open('email.json', 'w') as f:
    json.dump(email, f)
    
with open('phone.json', 'w') as f:
    json.dump(phone, f)

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

ner_model = legal.NerModel.pretrained('legner_contract_doc_parties_lg', 'en', 'legal/models')\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

ner_converter = legal.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setWhiteList(['EFFDATE', 'PARTY', 'ALIAS'])\
    .setReplaceLabels({'FORMER_NAME': 'PARTY'})\
    .setGreedyMode(True)

zero_shot_ner = legal.ZeroShotNerModel.pretrained("legner_roberta_zeroshot", "en", "legal/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("zero_shot_ner")\
    .setPredictionThreshold(0.1)\
    .setEntityDefinitions(
        {
            
            "ADDRESS":["Which address?", "Where is the location?"],
            "SIGNING_PERSON": ["Which person?", "What is the person name?"],
            "PARTY": ["Which LLC?", "Which Inc?", "Which PLC?", "Which Corp?"]
        })


zeroshot_ner_converter = legal.NerConverterInternal() \
    .setInputCols(["sentence", "token", "zero_shot_ner"])\
    .setOutputCol("zero_ner_chunk")\

ner_model2 = legal.NerModel.pretrained('legner_signers', 'en', 'legal/models')\
        .setInputCols(["sentence", "token", "embeddings"])\
        .setOutputCol("ner2")

ner_converter2 = nlp.NerConverter()\
        .setInputCols(["sentence","token","ner2"])\
        .setOutputCol("ner_chunk2")

alias_parser = legal.ContextualParserApproach() \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("alias")\
    .setJsonPath("alias.json") \
    .setPrefixAndSuffixMatch(False)\
    .setOptionalContextRules(True)\
    .setCaseSensitive(False)

email_parser = legal.ContextualParserApproach() \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("email")\
    .setJsonPath("email.json") \
    .setPrefixAndSuffixMatch(False)\
    .setOptionalContextRules(True)\
    .setCaseSensitive(False)

phone_parser = legal.ContextualParserApproach() \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("phone")\
    .setJsonPath("phone.json") \
    .setPrefixAndSuffixMatch(False)\
    .setOptionalContextRules(True)\
    .setCaseSensitive(False)

chunk_merger = legal.ChunkMergeApproach()\
    .setInputCols("email", "phone", "ner_chunk", "ner_chunk2","zero_ner_chunk", "alias")\
    .setOutputCol('merged_ner_chunks')

nlpPipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      ner_model,
      ner_converter,
      ner_model2,
      ner_converter2,
      zero_shot_ner,
      zeroshot_ner_converter,
      alias_parser,
      email_parser,
      phone_parser,
      chunk_merger])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ][OK!]
roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]
legner_contract_doc_parties_lg download started this may take some time.
[ | ][OK!]
legner_roberta_zeroshot download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
legner_signers download started this may take some time.
[ | ][ / ][ — ][OK!]

In [0]:
text = """
This Commercial Lease (this “Lease”) dated February 11, 2021, but made effective as of January 1, 2021 (the “Effective Date”), is made by and between 605 NASH, LLC, a California limited liability company (“Landlord”) and NANTKWEST, INC., a Delaware corporation (“Tenant”).

605 NASH, LLC,	 	NANTKWEST, inc.,
a California limited liability company	 	a Delaware corporation
 	 	 	 	 	 	 
 	 	 	 	 	 	 
By:	 	/s/ Charles Kenworthy	 	By:	 	/s/ Richard Adcock
Name: Charles N. Kenworthy	 	Name: Richard Adcock
Title:   Manager	 	Title:   CEO
 	 	 	 	 	 	 
Address:	 	Address:
9922 Jefferson Blvd.	 	3530 Johns Hopkins Court
Culver City, CA 90232	 	San Diego, CA 92121
Attention: Chuck Kenworthy	 	Attention: Chief Financial Officer
Email:
juan@johnsnowlabs.com
Telephone numbers:
304.123.333
304-123-333
+34 304-123-333
0034304123333
"""

In [0]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

# legal_ner
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------+---------+
chunk |ner_label|
+-----------------+---------+
February 11, 2021|EFFDATE |
January 1, 2021 |EFFDATE |
605 NASH, LLC |PARTY |
NANTKWEST, INC |PARTY |
NASH, LLC |PARTY |
NANTKWEST |PARTY |
+-----------------+---------+

In [0]:
# ner_signers
result.select(F.explode(F.arrays_zip(result.ner_chunk2.result, 
                                     result.ner_chunk2.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------+--------------+
chunk |ner_label |
+-----------------------+--------------+
605 |PARTY |
NASH, LLC, |PARTY |
NANTKWEST, INC |PARTY |
NASH, LLC, |PARTY |
NANTKWEST, inc |PARTY |
Charles Kenworthy |SIGNING_PERSON|
Richard Adcock |SIGNING_PERSON|
Charles N. Kenworthy |SIGNING_PERSON|
Richard Adcock |SIGNING_PERSON|
Manager |SIGNING_TITLE |
CEO |SIGNING_TITLE |
Chief Financial Officer|SIGNING_TITLE |
+-----------------------+--------------+

In [0]:
# zero_shot_ner
result.select(F.explode(F.arrays_zip(result.zero_ner_chunk.result, 
                                     result.zero_ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+---------------------------------------------+--------------+
chunk |ner_label |
+---------------------------------------------+--------------+
LLC |PARTY |
NANTKWEST |PARTY |
INC |PARTY |
Delaware corporation (“Tenant”). |PARTY |
Charles Kenworthy |SIGNING_PERSON|
Richard Adcock |SIGNING_PERSON|
Charles N. Kenworthy |SIGNING_PERSON|
Richard Adcock |SIGNING_PERSON|
CEO |SIGNING_PERSON|
9922 Jefferson Blvd |ADDRESS |
3530 Johns Hopkins Court |ADDRESS |
Culver City, CA 90232\t \tSan Diego, CA 92121|ADDRESS |
Chuck Kenworthy |SIGNING_PERSON|
Chief Financial Officer |SIGNING_PERSON|
+---------------------------------------------+--------------+

In [0]:
# merged_chunk
result.select(F.explode(F.arrays_zip(result.merged_ner_chunks.result, 
                                     result.merged_ner_chunks.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(n=50, truncate=False)

+---------------------------------------------+--------------+
chunk |ner_label |
+---------------------------------------------+--------------+
“Lease” |ALIAS |
February 11, 2021 |EFFDATE |
January 1, 2021 |EFFDATE |
“Effective Date” |ALIAS |
605 NASH, LLC |PARTY |
“Landlord” |ALIAS |
NANTKWEST, INC |PARTY |
Delaware corporation (“Tenant”). |PARTY |
NASH, LLC, |PARTY |
NANTKWEST, inc |PARTY |
Charles Kenworthy |SIGNING_PERSON|
Richard Adcock |SIGNING_PERSON|
Charles N. Kenworthy |SIGNING_PERSON|
Richard Adcock |SIGNING_PERSON|
Manager |SIGNING_TITLE |
CEO |SIGNING_TITLE |
9922 Jefferson Blvd |ADDRESS |
3530 Johns Hopkins Court |ADDRESS |
Culver City, CA 90232\t \tSan Diego, CA 92121|ADDRESS |
Chuck Kenworthy |SIGNING_PERSON|
Chief Financial Officer |SIGNING_TITLE |
juan@johnsnowlabs.com |EMAIL |
304.123.333 |PHONE |
304-123-333 |PHONE |
+34 304-123-333 |PHONE |
0034304123333 |PHONE |
+---------------------------------------------+--------------+

## Obfuscation mode

In the obfuscation mode **DeIdentificationModel** will replace sensitive entities with random values of the same type.

### Using external [Faker](https://faker.readthedocs.io/en/master/) library

In [0]:
!pip install faker

Collecting faker
 Downloading Faker-17.0.0-py3-none-any.whl (1.7 MB)
▏ | 10 kB 30.9 MB/s eta 0:00:01 |▍ | 20 kB 29.4 MB/s eta 0:00:01 |▋ | 30 kB 36.2 MB/s eta 0:00:01 |▊ | 40 kB 17.6 MB/s eta 0:00:01 |█ | 51 kB 20.3 MB/s eta 0:00:01 |█▏ | 61 kB 23.0 MB/s eta 0:00:01 |█▍ | 71 kB 21.3 MB/s eta 0:00:01 |█▌ | 81 kB 23.2 MB/s eta 0:00:01 |█▊ | 92 kB 24.9 MB/s eta 0:00:01 |██ | 102 kB 24.7 MB/s eta 0:00:01 |██▏ | 112 kB 24.7 MB/s eta 0:00:01 |██▎ | 122 kB 24.7 MB/s eta 0:00:01 |██▌ | 133 kB 24.7 MB/s eta 0:00:01 |██▊ | 143 kB 24.7 MB/s eta 0:00:01 |███ | 153 kB 24.7 MB/s eta 0:00:01 |███ | 163 kB 24.7 MB/s eta 0:00:01 |███▎ | 174 kB 24.7 MB/s eta 0:00:01 |███▌ | 184 kB 24.7 MB/s eta 0:00:01 |███▊ | 194 kB 24.7 MB/s eta 0:00:01 |███▉ | 204 kB 24.7 MB/s eta 0:00:01 |████ | 215 kB 24.7 MB/s eta 0:00:01 |████▎ | 225 kB 24.7 MB/s eta 0:00:01 |████▌ | 235 kB 24.7 MB/s eta 0:00:01 |████▋ | 245 kB 24.7 MB/s eta 0:00:01 |████▉ | 256 kB 24.7 MB/s eta 0:00:01 |█████ | 266 kB 24.7 MB/s eta 0:00:01 |█████▎ | 276 kB 24.7 MB/s eta 0:00:01 |█████▍ | 286 kB 24.7 MB/s eta 0:00:01 |█████▋ | 296 kB 24.7 MB/s eta 0:00:01 |█████▉ | 307 kB 24.7 MB/s eta 0:00:01 |██████ | 317 kB 24.7 MB/s eta 0:00:01 |██████▏ | 327 kB 24.7 MB/s eta 0:00:01 |██████▍ | 337 kB 24.7 MB/s eta 0:00:01 |██████▋ | 348 kB 24.7 MB/s eta 0:00:01 |██████▊ | 358 kB 24.7 MB/s eta 0:00:01 |███████ | 368 kB 24.7 MB/s eta 0:00:01 |███████▏ | 378 kB 24.7 MB/s eta 0:00:01 |███████▍ | 389 kB 24.7 MB/s eta 0:00:01 |███████▌ | 399 kB 24.7 MB/s eta 0:00:01 |███████▊ | 409 kB 24.7 MB/s eta 0:00:01 |████████ | 419 kB 24.7 MB/s eta 0:00:01 |████████▏ | 430 kB 24.7 MB/s eta 0:00:01 |████████▎ | 440 kB 24.7 MB/s eta 0:00:01 |████████▌ | 450 kB 24.7 MB/s eta 0:00:01 |████████▊ | 460 kB 24.7 MB/s eta 0:00:01 |█████████ | 471 kB 24.7 MB/s eta 0:00:01 |█████████ | 481 kB 24.7 MB/s eta 0:00:01 |█████████▎ | 491 kB 24.7 MB/s eta 0:00:01 |█████████▌ | 501 kB 24.7 MB/s eta 0:00:01 |█████████▊ | 512 kB 24.7 MB/s eta 0:00:01 |█████████▉ | 522 kB 24.7 MB/s eta 0:00:01 |██████████ | 532 kB 24.7 MB/s eta 0:00:01 |██████████▎ | 542 kB 24.7 MB/s eta 0:00:01 |██████████▌ | 552 kB 24.7 MB/s eta 0:00:01 |██████████▋ | 563 kB 24.7 MB/s eta 0:00:01 |██████████▉ | 573 kB 24.7 MB/s eta 0:00:01 |███████████ | 583 kB 24.7 MB/s eta 0:00:01 |███████████▏ | 593 kB 24.7 MB/s eta 0:00:01 |███████████▍ | 604 kB 24.7 MB/s eta 0:00:01 |███████████▋ | 614 kB 24.7 MB/s eta 0:00:01 |███████████▉ | 624 kB 24.7 MB/s eta 0:00:01 |████████████ | 634 kB 24.7 MB/s eta 0:00:01 |████████████▏ | 645 kB 24.7 MB/s eta 0:00:01 |████████████▍ | 655 kB 24.7 MB/s eta 0:00:01 |████████████▋ | 665 kB 24.7 MB/s eta 0:00:01 |████████████▊ | 675 kB 24.7 MB/s eta 0:00:01 |█████████████ | 686 kB 24.7 MB/s eta 0:00:01 |█████████████▏ | 696 kB 24.7 MB/s eta 0:00:01 |█████████████▍ | 706 kB 24.7 MB/s eta 0:00:01 |█████████████▌ | 716 kB 24.7 MB/s eta 0:00:01 |█████████████▊ | 727 kB 24.7 MB/s eta 0:00:01 |██████████████ | 737 kB 24.7 MB/s eta 0:00:01 |██████████████▏ | 747 kB 24.7 MB/s eta 0:00:01 |██████████████▎ | 757 kB 24.7 MB/s eta 0:00:01 |██████████████▌ | 768 kB 24.7 MB/s eta 0:00:01 |██████████████▊ | 778 kB 24.7 MB/s eta 0:00:01 |███████████████ | 788 kB 24.7 MB/s eta 0:00:01 |███████████████ | 798 kB 24.7 MB/s eta 0:00:01 |███████████████▎ | 808 kB 24.7 MB/s eta 0:00:01 |███████████████▌ | 819 kB 24.7 MB/s eta 0:00:01 |███████████████▊ | 829 kB 24.7 MB/s eta 0:00:01 |███████████████▉ | 839 kB 24.7 MB/s eta 0:00:01 |████████████████ | 849 kB 24.7 MB/s eta 0:00:01 |████████████████▎ | 860 kB 24.7 MB/s eta 0:00:01 |████████████████▌ | 870 kB 24.7 MB/s eta 0:00:01 |████████████████▋ | 880 kB 24.7 MB/s eta 0:00:01 |████████████████▉ | 890 kB 24.7 MB/s eta 0:00:01 |█████████████████ | 901 kB 24.7 MB/s eta 0:00:01 |█████████████████▏ | 911 kB 24.7 MB/s eta 0:00:01 |█████████████████▍ | 921 kB 24.7 MB/s eta 0:00:01 |█████████████████▋ | 931 kB 24.7 MB/s eta 0:00:01

In [0]:
from faker import Faker
fk = Faker()

In [0]:
# This is the obfuscation dict for the new entities
obs_lines = """CEO#SIGNING_TITLE
Chief Executive Officer#SIGNING_TITLE
Chief Legal Officer#SIGNING_TITLE
Chief Financial officer#SIGNING_TITLE
Legal Representative#SIGNING_TILE
"Alias"#ALIAS
"Alias"#ALIAS"""

for _ in range(25):
    add = fk.address().strip()
    for ad in add.split('\n'):
        obs_lines += f"\n{ad}#ADDRESS"
    obs_lines += f"\n{fk.name().strip()}#SIGNING_PERSON"
    obs_lines += f"\n{fk.date().strip()}#EFFDATE"
    obs_lines += f"\n{fk.company().strip()}#PARTY"
    obs_lines += f"\n{fk.phone_number().strip()}#PHONE"
    obs_lines += f"\n{fk.email().strip()}#EMAIL"

with open ('obfuscate.txt', 'w') as f:
    f.write(obs_lines)

In [0]:
# Previous Masking Annotators
#deid model with "entity_labels"
deid_entity_labels= legal.DeIdentification()\
    .setInputCols(["sentence", "token", "merged_ner_chunks"])\
    .setOutputCol("deidentified")\
    .setMode("mask")\
    .setMaskingPolicy("entity_labels")
    
#deid model with "same_length_chars"
deid_same_length= legal.DeIdentification()\
    .setInputCols(["sentence", "token", "merged_ner_chunks"])\
    .setOutputCol("masked_with_chars")\
    .setMode("mask")\
    .setMaskingPolicy("same_length_chars")

#deid model with "fixed_length_chars"
deid_fixed_length= legal.DeIdentification()\
    .setInputCols(["sentence", "token", "merged_ner_chunks"])\
    .setOutputCol("masked_fixed_length_chars")\
    .setMode("mask")\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)


In [0]:
# Obfuscation with Faker
obfuscation = legal.DeIdentification()\
    .setInputCols(["sentence", "token", "merged_ner_chunks"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate")\
    .setObfuscateDate(True)\
    .setObfuscateRefFile('obfuscate.txt')\
    .setObfuscateRefSource("both")

nlpPipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      ner_model,
      ner_converter,
      ner_model2,
      ner_converter2,
      zero_shot_ner,
      zeroshot_ner_converter,
      alias_parser,
      email_parser,
      phone_parser,
      chunk_merger,
      deid_entity_labels,
      deid_same_length,
      deid_fixed_length,
      obfuscation])

obfuscation_model = nlpPipeline.fit(empty_data)

In [0]:
text = """This Commercial Lease (this “Lease”) dated February 11, 2021, but made effective as of January 1, 2021 (the “Effective Date”), is made by and between 605 NASH, LLC, a California limited liability company (“Landlord”) and NANTKWEST, INC., a Delaware corporation (“Tenant”).

605 NASH, LLC,	 	NANTKWEST, inc.,
a California limited liability company	 	a Delaware corporation
 	 	 	 	 	 	 
 	 	 	 	 	 	 
By:	 	/s/ Charles Kenworthy	 	By:	 	/s/ Richard Adcock
Name: Charles N. Kenworthy	 	Name: Richard Adcock
Title:   Manager	 	Title:   CEO
 	 	 	 	 	 	 
Address:	 	Address:
9922 Jefferson Blvd.	 	3530 Johns Hopkins Court
Culver City, CA 90232	 	San Diego, CA 92121
Attention: Chuck Kenworthy	 	Attention: Chief Financial Officer cfo@johnkopkins.com (0031) 913-123"""

In [0]:
result = obfuscation_model.transform(spark.createDataFrame([[text]]).toDF("text"))
print("\n".join(result.select('obfuscated.result').collect()[0].result))

This Commercial Lease (this "Alias") dated 1996-07-13, but made effective as of 1994-09-25 (the "Alias"), is made by and between Cooper-Adams, a California limited liability company ("Alias") and Hernandez, Crosby and Hunt., a Moore, Clay and Kelly corporation ("Alias").
605 Howard PLC Chapman-Miller.,
a California limited liability company a Delaware corporation
By: /s/ Shannon Johnson By: /s/ Gregory Lucas
Name: Gregory Lucas Name: Gregory Lucas
Title: CEO Title: Chief Executive Officer
 
Address: Address:
154 Taylor Islands.
 70453 Eric Village
Unit 5632 Box 4484
Attention: Emma Goodwin Attention: Chief Executive Officer Cris@yahoo.com (9637 3075

## Using Light Pipelines

In [0]:
light_model = nlp.LightPipeline(obfuscation_model)
annotated_text = light_model.annotate(text)
print("\n".join(annotated_text['deidentified']))

This Commercial Lease (this <ALIAS>) dated <EFFDATE>, but made effective as of <EFFDATE> (the <ALIAS>), is made by and between <PARTY>, a California limited liability company (<ALIAS>) and <PARTY>., a <PARTY> corporation (<ALIAS>).
605 <PARTY> <PARTY>.,
a California limited liability company a Delaware corporation
By: /s/ <SIGNING_PERSON> By: /s/ <SIGNING_PERSON>
Name: <SIGNING_PERSON> Name: <SIGNING_PERSON>
Title: <SIGNING_TITLE> Title: <SIGNING_TITLE>
 
Address: Address:
<ADDRESS>.
 <ADDRESS>
<ADDRESS>
Attention: <SIGNING_PERSON> Attention: <SIGNING_TITLE> <EMAIL> (<PHONE>

In [0]:
print("\n".join(annotated_text['obfuscated']))

This Commercial Lease (this "Alias") dated 1996-07-13, but made effective as of 1994-09-25 (the "Alias"), is made by and between Cooper-Adams, a California limited liability company ("Alias") and Hernandez, Crosby and Hunt., a Moore, Clay and Kelly corporation ("Alias").
605 Howard PLC Chapman-Miller.,
a California limited liability company a Delaware corporation
By: /s/ Shannon Johnson By: /s/ Gregory Lucas
Name: Gregory Lucas Name: Gregory Lucas
Title: CEO Title: Chief Executive Officer
 
Address: Address:
154 Taylor Islands.
 70453 Eric Village
Unit 5632 Box 4484
Attention: Emma Goodwin Attention: Chief Executive Officer Cris@yahoo.com (9637 3075

## Shifting Days

We use the `medical.DocumentHashCoder()` annotator to determine shifting days. This annotator gets the hash of the specified column and creates a new document column containing day shift information. And then, the `medical.DeIdentification()` annotator deidentifies this new doc. We should set the seed parameter to hash consistently.

In [0]:
import pandas as pd

data = pd.DataFrame(
    {'DocumentID' : ['A001', 'A001', 'A002', 'A002'],
     'text' : ['Chris Brown was arrested on 10/02/2022', 
               'Mark White has bought a stock on 02/28/2020', 
               'John has bought a house on 03/15/2022',
               'John Moore was discharged on 12/31/2022'
              ]
    }
)

my_input_df = spark.createDataFrame(data)

my_input_df.show(truncate = False)

+----------+-------------------------------------------+
DocumentID|text |
+----------+-------------------------------------------+
A001 |Chris Brown was arrested on 10/02/2022 |
A001 |Mark White has bought a stock on 02/28/2020|
A002 |John has bought a house on 03/15/2022 |
A002 |John Moore was discharged on 12/31/2022 |
+----------+-------------------------------------------+

### Shifting days according to the ID column

We use the `legal.DocumentHashCoder()` annotator to determine shifting days. This annotator gets the hash of the specified column and creates a new document column containing day shift information. And then, the `legal.DeIdentification()` annotator deidentifies this new doc. We should set the seed parameter to hash consistently.

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentHasher = legal.DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setPatientIdColumn("DocumentID")\
    .setRangeDays(100)\
    .setNewDateShift("shift_days")\
    .setSeed(100)


# sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
#     .setInputCols(["document2"])\
#     .setOutputCol("sentence")
#     #.setCustomBounds(["\n\n"])

tokenizer = nlp.Tokenizer()\
    .setInputCols(["document2"])\
    .setOutputCol("token")

embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document2", "token"]) \
    .setOutputCol("embeddings")

legal_ner = legal.NerModel.pretrained('legner_deid', "en", "legal/models")\
    .setInputCols(["document2", "token", "embeddings"]) \
    .setOutputCol("ner") 
    #.setLabelCasing("upper")

ner_converter = legal.NerConverterInternal() \
    .setInputCols(["document2", "token", "ner"])\
    .setOutputCol("ner_chunk") # "ALIAS" are secondary names of companies, so let's extract them also as PARTY


deid = legal.DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("obfuscate") \
    .setObfuscateDate(True) \
    .setDateTag("DATE") \
    .setLanguage("en") \
    .setObfuscateRefSource('faker') \
    .setUseShifDays(True)\
    .setRegion('us')

pipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      documentHasher,
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter,
      deid])

empty_data = spark.createDataFrame([["", ""]]).toDF("text", "DocumentID")

pipeline_model = pipeline.fit(empty_data)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]
legner_deid download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
output = pipeline_model.transform(my_input_df)

output.select('DocumentID','text', 'deidentified.result').show(truncate = False)

+----------+-------------------------------------------+-------------------------------------------+
DocumentID|text |result |
+----------+-------------------------------------------+-------------------------------------------+
A001 |Chris Brown was arrested on 10/02/2022 |[<PERSON> was arrested on 11/05/2022] |
A001 |Mark White has bought a stock on 02/28/2020|[<PERSON> has bought a stock on 03/24/2020]|
A002 |John has bought a house on 03/15/2022 |[<PERSON> has bought a house on 04/10/2022]|
A002 |John Moore was discharged on 12/31/2022 |[<PERSON> was discharged on 01/20/2023] |
+----------+-------------------------------------------+-------------------------------------------+

### Shifting days according to specified values

Instead of shifting days according to ID column, we can specify shifting values with another column.

```python
documentHasher = legal.DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setDateShiftColumn("dateshift")\
```

In [0]:
data = pd.DataFrame(
    {'DocumentID' : ['A001', 'A001', 'A002', 'A002'],
     'text' : ['Chris Brown was arrested on 10/02/2019', 
               'Mark White has bought a stock on 02/28/2020', 
               'John has bought a house on 03/15/2022',
               'John Moore was discharged on 12/31/2022'
                            ],
     'dateshift' : ['5', '5', '10', '10']
    }
)


my_input_df = spark.createDataFrame(data)

my_input_df.show(truncate = False)

+----------+-------------------------------------------+---------+
DocumentID|text |dateshift|
+----------+-------------------------------------------+---------+
A001 |Chris Brown was arrested on 10/02/2019 |5 |
A001 |Mark White has bought a stock on 02/28/2020|5 |
A002 |John has bought a house on 03/15/2022 |10 |
A002 |John Moore was discharged on 12/31/2022 |10 |
+----------+-------------------------------------------+---------+

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentHasher = legal.DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setDateShiftColumn("dateshift")\


# sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
#     .setInputCols(["document2"])\
#     .setOutputCol("sentence")
#     #.setCustomBounds(["\n\n"])

tokenizer = nlp.Tokenizer()\
    .setInputCols(["document2"])\
    .setOutputCol("token")

embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document2", "token"]) \
    .setOutputCol("embeddings")

legal_ner = legal.NerModel.pretrained('legner_deid', "en", "legal/models")\
    .setInputCols(["document2", "token", "embeddings"]) \
    .setOutputCol("ner") 
    #.setLabelCasing("upper")

ner_converter = legal.NerConverterInternal() \
    .setInputCols(["document2", "token", "ner"])\
    .setOutputCol("ner_chunk") # "ALIAS" are secondary names of companies, so let's extract them also as PARTY

obfuscation = legal.DeIdentification()\
    .setInputCols(["document2", "token", "ner_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("obfuscate") \
    .setObfuscateDate(True) \
    .setDateTag("DATE") \
    .setLanguage("en") \
    .setObfuscateRefSource('faker') \
    .setUseShifDays(True)\
    .setRegion('us')

pipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      documentHasher,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter,
      obfuscation])

empty_data = spark.createDataFrame([["", "", ""]]).toDF("text", "DocumentID", "dateshift")

pipeline_model = pipeline.fit(empty_data)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]
legner_deid download started this may take some time.
[ | ][OK!]

In [0]:
output = pipeline_model.transform(my_input_df)

output.select('text', 'dateshift', 'deidentified.result').show(truncate = False)

+-------------------------------------------+---------+-------------------------------------------+
text |dateshift|result |
+-------------------------------------------+---------+-------------------------------------------+
Chris Brown was arrested on 10/02/2019 |5 |[<PERSON> was arrested on 10/07/2019] |
Mark White has bought a stock on 02/28/2020|5 |[<PERSON> has bought a stock on 03/04/2020]|
John has bought a house on 03/15/2022 |10 |[<PERSON> has bought a house on 03/25/2022]|
John Moore was discharged on 12/31/2022 |10 |[<PERSON> was discharged on 01/10/2023] |
+-------------------------------------------+---------+-------------------------------------------+

### Masking Unnormalized Date Formats

`setUnnormalizedDateMode()` parameter is used to mask the DATE entities that can not be normalized. In the example below, please check `03Apr2022` which couldn't be normalized and it is masked in the output.

In [0]:
data = pd.DataFrame(
    {'DocumentID' : ['A001', 'A001', 'A002', 'A002'],
     'text' : ['Chris Brown was arrested on 10/02/2022', 
               'Mark White has bought a stock on 02/28/2020', 
               'John has bought a house on 03Apr2022',
               'John Moore was discharged on 12/31/2022'
                            ],
     'dateshift' : ['5', '5', '10', '10']
    }
)

my_input_df = spark.createDataFrame(data)


documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

documentHasher = legal.DocumentHashCoder()\
    .setInputCols("document")\
    .setOutputCol("document2")\
    .setDateShiftColumn("dateshift")\


# sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
#     .setInputCols(["document2"])\
#     .setOutputCol("sentence")
#     #.setCustomBounds(["\n\n"])

tokenizer = nlp.Tokenizer()\
    .setInputCols(["document2"])\
    .setOutputCol("token")

embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document2", "token"]) \
    .setOutputCol("embeddings")

legal_ner = legal.NerModel.pretrained('legner_deid', "en", "legal/models")\
    .setInputCols(["document2", "token", "embeddings"]) \
    .setOutputCol("ner") 
    #.setLabelCasing("upper")

ner_converter = legal.NerConverterInternal() \
    .setInputCols(["document2", "token", "ner"])\
    .setOutputCol("ner_chunk") # "ALIAS" are secondary names of companies, so let's extract them also as PARTY

obfuscation = legal.DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("obfuscate") \
    .setObfuscateDate(True) \
    .setDateTag("DATE") \
    .setLanguage("en") \
    .setObfuscateRefSource('faker') \
    .setUseShifDays(True)\
    .setRegion('us')\
    .setUnnormalizedDateMode("mask")

pipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      documentHasher,
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter,
      obfuscation])


output = pipeline.fit(my_input_df).transform(my_input_df)

output.select('text', 'dateshift', 'deidentified.result').show(truncate = False)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]
legner_deid download started this may take some time.
[ | ][OK!]
+-------------------------------------------+---------+-------------------------------------------+
text |dateshift|result |
+-------------------------------------------+---------+-------------------------------------------+
Chris Brown was arrested on 10/02/2022 |5 |[<PERSON> was arrested on 11/27/2022] |
Mark White has bought a stock on 02/28/2020|5 |[<PERSON> has bought a stock on 04/12/2020]|
John has bought a house on 03Apr2022 |10 |[<PERSON> has bought a house on <DATE>] |
John Moore was discharged on 12/31/2022 |10 |[<PERSON> was discharged on 02/07/2023] |
+-------------------------------------------+---------+-------------------------------------------+

# Structured Deidentification

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/legal-nlp/data/hipaa-table-001.txt

df = spark.read.format("csv") \
    .option("sep", "\t") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .load("hipaa-table-001.txt")

df.show(truncate=False)

--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-2861126742754184> in <module> 
 1 get_ipython ( ) . system ( 'wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/legal-nlp/data/hipaa-table-001.txt' ) 
 2 
 ----> 3 df = spark . read . format ( "csv" ) \ 
 4 . option ( "sep" , "\t" ) \ 
 5 . option ( "inferSchema" , "true" ) \ 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 156 self . options ( ** options ) 
 157 if isinstance ( path , str ) : 
 --> 158 return self . _df ( self . _jreader . load ( path ) ) 
 159 elif path is not None : 
 160 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 IllegalArgumentException : Path must be absolute: hipaa-table-001.txt

In [0]:
obfuscator = legal.StructuredDeidentification(spark,{"NAME":"PATIENT","AGE":"AGE"}, obfuscateRefSource = "faker")
obfuscator_df = obfuscator.obfuscateColumns(df)
obfuscator_df.show(truncate=False)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2861126742754185> in <module> 
 1 obfuscator = legal . StructuredDeidentification ( spark , { "NAME" : "PATIENT" , "AGE" : "AGE" } , obfuscateRefSource = "faker" ) 
 ----> 2 obfuscator_df = obfuscator . obfuscateColumns ( df ) 
 3 obfuscator_df . show ( truncate = False ) 

 NameError : name 'df' is not defined

In [0]:
obfuscator_unique_ref_test = '''Will Perry#CLIENT
John Smith#CLIENT
Marvin MARSHALL#CLIENT
Hubert GROGAN#CLIENT
ALTHEA COLBURN#CLIENT
Kalil AMIN#CLIENT
Inci FOUNTAIN#CLIENT
Jackson WILLE#CLIENT
Jack SANTOS#CLIENT
Mahmood ALBURN#CLIENT
Marnie MELINGTON#CLIENT
Aysha GHAZI#CLIENT
Maryland CODER#CLIENT
Darene GEORGIOUS#CLIENT
Shelly WELLBECK#CLIENT
Min Kun JAE#CLIENT
Thomson THOMAS#CLIENT
Christian SUDDINBURG#CLIENT
Aberdeen#CITY
Louisburg St#STREET
France#LOC
5552312#PHONE
Calle del Libertador#ADDRESS
111#ID
20#AGE
30#AGE
40#AGE
50#AGE
60#AGE
'''

with open('obfuscator_unique_ref_test.txt', 'w') as f:
  f.write(obfuscator_unique_ref_test)

In [0]:
# obfuscateRefSource = "file"

obfuscator = legal.StructuredDeidentification(spark,{"NAME":"CLIENT","AGE":"AGE"}, 
                                        obfuscateRefFile = "/content/obfuscator_unique_ref_test.txt",
                                        obfuscateRefSource = "file",
                                        columnsSeed={"NAME": 23, "AGE": 23})
obfuscator_df = obfuscator.obfuscateColumns(df)
obfuscator_df.select("NAME","AGE").show(truncate=False)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2861126742754187> in <module> 
 5 obfuscateRefSource = "file" , 
 6 columnsSeed={"NAME": 23, "AGE": 23})
 ----> 7 obfuscator_df = obfuscator . obfuscateColumns ( df ) 
 8 obfuscator_df . select ( "NAME" , "AGE" ) . show ( truncate = False ) 

 NameError : name 'df' is not defined

We can **shift n days** in the structured deidentification through "days" parameter when the column is a Date.

In [0]:
df = spark.createDataFrame([
            ["Juan García", "13/02/1977", "711 Nulla St.", "140", "673 431234"],
            ["Will Smith", "23/02/1977", "1 Green Avenue.", "140", "+23 (673) 431234"],
            ["Pedro Ximénez", "11/04/1900", "Calle del Libertador, 7", "100", "912 345623"]
        ]).toDF("NAME", "DOB", "ADDRESS", "SBP", "TEL")
df.show(truncate=False)

+-------------+----------+-----------------------+---+----------------+
NAME |DOB |ADDRESS |SBP|TEL |
+-------------+----------+-----------------------+---+----------------+
Juan García |13/02/1977|711 Nulla St. |140|673 431234 |
Will Smith |23/02/1977|1 Green Avenue. |140|+23 (673) 431234|
Pedro Ximénez|11/04/1900|Calle del Libertador, 7|100|912 345623 |
+-------------+----------+-----------------------+---+----------------+

In [0]:
obfuscator = legal.StructuredDeidentification(spark=spark, 
                                        columns={"NAME": "ID", "DOB": "DATE"},
                                        columnsSeed={"NAME": 23, "DOB": 23},
                                        obfuscateRefSource="faker",
                                        days=5
                                         )

In [0]:
result = obfuscator.obfuscateColumns(df)
result.show(truncate=False)

+----------+------------+-----------------------+---+----------------+
NAME |DOB |ADDRESS |SBP|TEL |
+----------+------------+-----------------------+---+----------------+
[N2649912]|[18/02/1977]|711 Nulla St. |140|673 431234 |
[W466004] |[28/02/1977]|1 Green Avenue. |140|+23 (673) 431234|
[M403810] |[16/04/1900]|Calle del Libertador, 7|100|912 345623 |
+----------+------------+-----------------------+---+----------------+

# Save the Pipeline and Use it from Your Local

In [0]:
model.write().overwrite().save('/dbfs/pipeline_deid')

In [0]:
# from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = nlp.PretrainedPipeline.from_disk("/dbfs/pipeline_deid")

In [0]:
data = spark.createDataFrame([[text]]).toDF("text")

In [0]:
deid_pipeline.model.stages

Out[58]: [DocumentAssembler_2bbb61f3930c,
 SentenceDetectorDLModel_8aaebf7e098e,
 REGEX_TOKENIZER_8c6889fc5b79,
 ROBERTA_EMBEDDINGS_b915dff90901,
 LegalNerModel_5eb62585382d,
 NER_CONVERTER_e9a9d3178ecf,
 MedicalNerModel_2b2f0f671f99,
 NerConverter_2e78104ae1fd,
 ZeroShotRobertaNer_5d06c0297d21,
 NER_CONVERTER_465376510d4b,
 CONTEXTUAL-PARSER_a7f2ac9a9a64,
 CONTEXTUAL-PARSER_9914955b40c6,
 CONTEXTUAL-PARSER_2a837a58ea1a,
 MERGE_e3b4ee018e70]

In [0]:
deid_pipeline.model.transform(data).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| ner2| ner_chunk2| zero_shot_ner| zero_ner_chunk| alias| email| phone| merged_ner_chunks|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
This Commercial L...|[{document, 0, 76...|[{document, 0, 27...|[{token, 0, 3, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 43, 59, ...|[{named_entity, 0...|[{chunk, 150, 152...|[{named_entity, 0...|[{chunk, 160, 162...|[{chunk, 28, 34, ...|[{chunk, 727, 745...|[{chunk, 748, 760...|[{chunk, 28, 34, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+